In [1]:
#dependencies
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
#from bayes_opt import BayesianOptimization

In [2]:
#Parameter values

#from Farquhar 1980
#turnover rate for carboxylation
kc = 2.5
#turnover rate for oxygenation
ko = .21*kc
#light intensity
I = 150
#total enzyme concentration
Et = 87.2
#CO2 concentration
C = 230
#O2 concentration
O = 210

#from Ye 2020
alpha = .295
beta = 2.42*(10**(-3))
gamma = 1.26*(10**(-4))

#couldnt find turnover # for RuBP so am setting kr, etc. = kc
kr = kc
kprod = kc
kE = kc

#also guesses here
kERC = .9*kc
kERO = .9*ko

kappac = 2/3
kappar = 1/3

In [3]:
#model for PGA reduction vs RuBP *with Ye model for light
def jprod(R, C = 230, O =210, kc = kc, ko = ko, kappac = kappac, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    jph = I*alpha*(1-beta*I)/(1+gamma*I)
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    jprod = ((1/kE)+(1/kappac*jph)+(1/jpga)-(1/(jpga+jph)))**(-1)
    return jprod

In [4]:
#plotting jprod against RuBP
@interact(Rsteps= (0, 5000, 10), Et = (0, 1000, 10), kr  = (.1, 10, .1))
def simulateR(C = 230, O =210, kc = kc, ko = ko, kappac=kappac, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = 10, kE = kE, kERO=kERO, kERC = kERC, Rsteps=300):
    vcr = []
    for k in range(1,Rsteps):
        vcr.append(jprod(k, C=C, O=O, kc = kc, ko = ko, kappac=kappac, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC))
    ksum = ko*O/kERO + kc*C/kERC
    
    plt.plot(range(1,Rsteps), vcr)
    plt.xlabel('RuBP')
    plt.ylabel('jprod')
    plt.show()

interactive(children=(IntSlider(value=230, description='C', max=690, min=-230), IntSlider(value=210, descripti…

In [5]:
#plotting jprod against irradiance, still with Ye model
@interact(Isteps= (0, 5000, 10), Et = (0, 1000, 10), kr  = (.1, 10, .1))
def simulateI(R= 100, C = 230, O =210, kc = kc, ko = ko, kappac=kappac, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = 10, kE = kE, kERO=kERO, kERC = kERC, Isteps=1000):
    vcI = []
    for i in range(1,Isteps):
        vcI.append(jprod(R=R, C=C, O=O, kc = kc, ko = ko, kappac=kappac, I = i, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC))
    
    plt.plot(range(1,Isteps), vcI)
    plt.xlabel('Irradiance')
    plt.ylabel('jprod')
    plt.show()

interactive(children=(IntSlider(value=100, description='R', max=300, min=-100), IntSlider(value=230, descripti…

In [6]:
#various models for irradiance
#parameters
zun = .21
zdo = .33

In [7]:
#Ye model
def Ye(I, alpha = alpha, beta = beta, gamma = gamma):
    jph = alpha*(1-beta*I)*I/(1+gamma*I)
    return jph

In [8]:
#Ye model with irradiance dependance
def altYe(I, alpha = alpha, beta = beta, zun = zun, zdo = zdo, gamma = gamma):
    frac = beta*I/(zun*I+zdo)
    jph = I*alpha*(1-frac)/(1+gamma*I)
    return jph

In [9]:
#circuit formulation
Imax = 700
Rc = .01
def circuit(I, Imax=Imax, Rc=Rc):
    jph = I/((I/Imax)+Rc)
    return jph

In [10]:
#Various models as functions of irradiance
@interact()
def simulatejph(irange=5000, zun=zun, zdo=zdo, Imax = Imax, Rc=Rc, alpha = alpha, beta = beta, gamma = gamma):
    jphylist = []
    jphalist = []
    jphblist = []
    for i in range(1,irange):
        jphylist.append(Ye(i, alpha = alpha, beta = beta, gamma = gamma))
        jphalist.append(altYe(i, alpha = alpha, beta = beta, zun = zun, zdo = zdo, gamma = gamma))
        jphblist.append(circuit(i, Imax, Rc))
    plt.plot(range(1,irange), jphylist, label='Ye model')
    plt.plot(range(1, irange), jphalist, label = 'model A')
    plt.plot(range(1,irange), jphblist, label = 'model B')
    plt.xlabel('Irradiance')
    plt.ylabel('jph')
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=5000, description='irange', max=15000, min=-5000), FloatSlider(value=0.2…

In [11]:
#RuBP regeneration
alphaa = 1
alphab = 1
def dRdtalt(R, C = 230, O =210, kc = kc, ko = ko, kappac = kappac, kappar = kappar, alphaa = alphaa, alphab = alphab, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    jph = I*alpha*(1-beta*I)/(1+gamma*I)
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    kEa = alphaa*kc
    kEb = alphab*kc
    jpc = kappac*jph
    jpr = kappar*jph
    jprod = (5/6)*((1/kEa)+(1/(kappac*jpc))+(1/jpga)-(1/(jpga+jpc)))**(-1)
    jrubp = (1/kEb + 1/(kappar*jpr) + 1/jprod - 1/(jpr+jprod))**(-1)
    dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + jrubp
    return dRdt

In [12]:
@interact(alpha = (0,100,.1))
#note that R does not scale to realistic values, likely due to wrong parameterization
def simulatealtR(Ro = .0014, tsteps = 500, dt = .001, kappac = kappac, kappar = kappar, C = 230, O =210, kc = kc, ko = ko, alphaa = alphaa, alphab=alphab, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    R = np.zeros(tsteps)
    R[0] = Ro
    for t in range(tsteps-1):
        R[t+1] = R[t] + dRdtalt(R[t], C=C, alpha = alpha, alphaa = alphaa, alphab = alphab)*dt
    plt.plot(range(tsteps),R, label = 'RuBP')
    plt.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(FloatSlider(value=0.0014, description='Ro', max=0.0042, min=-0.0014), IntSlider(value=50…

In [13]:
#params
kappaa = 4
m = .2
#these params are made up. Might need high m for NPQ to make a difference
kf = .2
knpq = .1
kpq = .7

In [14]:

#Alternative light model: dynamical system with balancing fluxes
def jprodalt(pq, I, R):
    jph = pq
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    kEa = alphaa*kc
    kEb = alphab*kc
    jpc = kappac*jph
    jpr = kappar*jph
    jprod = ((1/kEa)+(1/(kappac*jpc))+(1/jpga)-(1/(jpga+kappac*jpc)))**(-1)
    jprodr = (5/6)*jprod
    jrubp = (1/kEb + 1/(kappar*jpr) + 1/jprodr - 1/(kappar*jpr+jprodr))**(-1)
    dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + jrubp
    return jprod, dRdt, jpga

In [15]:
def jpq(pq, I, R, jf=.1*I, m=.2, jsyn=1.5):
    jp, jr, jpga = jprodalt(pq, I, R)
    jadp = jp + jr
    jatp = (1/jsyn + 1/(kappaa*pq) + 1/jadp - 1/(kappaa*pq + jadp))**(-1)
    jnpq = m*(pq - jatp)
    jpq = I*kpq/(kpq+kf+knpq*jnpq)
    return jp, jpq, jr, jnpq, jpga, jatp, jadp

In [16]:
#light input
def logistic(Io, Imax, x, kl):
    f = Imax/(1+np.exp(-kl*(x-Io)))
    return f

In [25]:
@interact(pqo = (0,100,.1), Ro=(0.001,.002,.0001), time = (0,10,1))
def integrated(pqo, Ro, m=m, ik = 5000, Io=100, Imax=500, kl = 1, time=1000, dt = .001, C = 230, O =210, kc = kc, ko = ko, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    tsteps = np.linspace(0, time, time*10)
    jplist = np.zeros(len(tsteps))
    jpqlist = np.zeros(len(tsteps))
    Rlist = np.zeros(len(tsteps))
    npqlist = np.zeros(len(tsteps))
    jpgalist = np.zeros(len(tsteps))
    atplist = np.zeros(len(tsteps))
    adplist = np.zeros(len(tsteps))
    jpqlist[0] = pqo
    Rlist[0] = Ro
    jplist[0] = jprodalt(pqo, ik, Ro)[0]
    jpgalist[0] = jprodalt(pqo, ik, Ro)[2]
    ilist = [ik for t in tsteps]
    for t in range(len(tsteps)-1):
        jp, jpqu, jr, jnpq, jpga, jatp, jadp = jpq(jpqlist[t], ilist[t], Rlist[t], m=m)
        Rlist[t+1] = Rlist[t] + jr*dt
        jplist[t+1] = jp
        jpqlist[t+1] = jpqu
        atplist[t+1] = jatp
        adplist[t+1] = jadp
        npqlist[t] = jnpq
        jpgalist[t] = jpga
    atplist[0] = atplist[1]
    adplist[0] = adplist[1]
    fig, axs = plt.subplots(3)
    fig.set_figheight(5)
    fig.set_figwidth(10)
    axs[0].plot(tsteps, jpqlist, label='Jpq')
    axs[1].plot(tsteps, Rlist, label = 'RuBP', color='grey')
    axs[0].plot(tsteps, ilist, label='Irradiance')
    axs[1].plot(tsteps, jplist, label = 'jprod', color='red')
    jpgalist[-1] = jpgalist[-2]
    axs[1].plot(tsteps, jpgalist, label = 'jpga')
    npqlist[-1] = npqlist[-2]
    axs[0].plot(tsteps, npqlist, label='NPQ')
    axs[2].plot(tsteps, atplist, label='jATP', color='brown')
    axs[2].plot(tsteps, adplist, label='jADP', color='pink')
    axs[2].legend()
    axs[0].legend(loc='upper right')
    axs[1].legend(loc='upper right')  
    axs[0].set(xlabel='time')
    axs[1].set(xlabel='time')
    axs[2].set(xlabel='time')

interactive(children=(FloatSlider(value=50.0, description='pqo'), FloatSlider(value=0.0015, description='Ro', …

In [91]:
def jATP(atp, pq, R, jsyn = 1.5, kappaa = kappaa, kp = 100, kpr = 2, C=C, O=O, Et=Et):
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    atpc = 2*atp/3
    atpr = atp/3
    jprod = (1/kp + 1/atpc + 1/jpga - 1/(jpga+atpc))**(-1)
    jprodr = (5/6)*jprod
    jrubp = (1/kpr + 1/(atpr) + 1/jprodr - 1/(jprodr+atpr))**(-1)
    dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + jrubp
    jadp = 2*(1/jprod + 1/jrubp - 1/(jrubp+jprod))**(-1)
    atpf = (1/jsyn + 1/(kappaa*pq) + 1/jadp - 1/(kappaa*pq + jadp))**(-1)
    return atpf, jadp, dRdt, jprod, jpga

In [92]:
def jpqalt(I, pq, atp, kf=kf, kpq=kpq, m=m, knpq=knpq):
    f = m*(pq-atp)
    jpq = I*(kpq/(kpq+kf+knpq*f))
    return jpq, f

In [93]:
@interact()
def intsim(pqo = 1, Ro=1, atpo=100, ik=1, dt = .001, time=2, jsyn = 1000, kappaa = kappaa, kp = 2000, kpr = 2, kf=0, kpq=kpq, m=0, knpq=knpq, C=C, O=O, Et=Et):
    tsteps = np.linspace(0, time, time*100)
    xtsteps = tsteps[1:len(tsteps)]
    jplist = np.zeros(len(xtsteps))
    jpqlist = np.zeros(len(tsteps))
    Rlist = np.zeros(len(tsteps))
    npqlist = np.zeros(len(xtsteps))
    atplist = np.zeros(len(tsteps))
    adplist = np.zeros(len(xtsteps))
    jpqlist[0] = pqo
    Rlist[0] = Ro
    atplist[0] = atpo
    pgalist = np.zeros(len(xtsteps))
    ilist = [ik for t in tsteps]
    for t in range(len(tsteps)-1):
        atpf, jadp, dRdt, jprod, jpga = jATP(atplist[t], jpqlist[t], Rlist[t], jsyn = jsyn, kappaa = kappaa, kp = kp, kpr = kpr, C=C, O=O, Et=Et)
        atplist[t+1] = atpf
        adplist[t] = jadp
        Rlist[t+1] = Rlist[t] + dRdt*dt
        jplist[t] = jprod
        pgalist[t] = jpga
        p, f = jpqalt(ilist[t], jpqlist[t], atplist[t+1], kf=kf, kpq=kpq, m=m, knpq=knpq)
        jpqlist[t+1] = p
        npqlist[t] = f
    plt.rcParams["figure.figsize"] = (20,3)
    #plt.plot(tsteps, jpqlist, label='Jpq')
    #plt.plot(tsteps, Rlist,  'r--', label = 'RuBP')
    #plt.plot(tsteps, ilist, label='Irradiance')
    #plt.scatter(xtsteps, jplist, label = 'jprod', color='red')
    #plt.plot(xtsteps, npqlist, label='NPQ')
    plt.plot(tsteps, atplist, label='jATP', color='brown')
    #plt.scatter(xtsteps, adplist, label='jADP', color='pink')
    plt.plot(xtsteps, pgalist, label = 'jpga', color='purple')
    plt.legend()
    plt.show()
    pga = pgalist[0]
    print("pga ",pga)
    atpc = 2*atplist[0]/3
    print("2/3 atp", atpc)
    plus = (1/kp + 1/atpc + 1/pga)
    print("atp + pga", plus)
    minus = (1/(atpc+pga))
    print("sub", minus)

interactive(children=(IntSlider(value=1, description='pqo', max=3, min=-1), IntSlider(value=1, description='Ro…

In [144]:
#alternative model with continuous development of ATP
kAC = 2
kAR = kAC/3
kI = 2
EprT = 80
ErgT = 80
kATP = 2
jsyn = 1.5
kpg = 2
Esyn = 80

def dR(dRu, dADPr):
    dR = -dRu + dADPr
    return dR
def dPGA(R, dADPc):
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    dPGA = (2*jcA+1.5*joA) - dADPc
    dRu = jcA + joA 
    return dPGA, dRu
def dADP(ATP, PGA, Int):
    rfrac = (1/kr + 1/(kAR*ATP) + 1/(kI*Int) - 1/(kAR*ATP + kI*Int))
    dADPr = ErgT*(1/rfrac)
    cfrac = (1/kI + 1/(kAC*ATP) + 1/(kpg*PGA)- 1/(kpg*PGA+kAC*ATP))
    dADPc = EprT*(1/cfrac)
    dInt = dADPc - dADPr
    dADP = dADPc + dADPr
    return dADP, dInt, dADPr, dADPc
def dATP(ADP, dADP, jpq):
    fATP = kATP*ADP
    dATP = Esyn/(1/jsyn + 1/fATP + 1/jpq - 1/(jpq+fATP)) - dADP
    return dATP
def PQ(pqt, jatp):
    f = m*(pqt-jatp)
    jpq = I*(kpq/(kpq+kf+knpq*f))
    return jpq, f

In [163]:
@interact()
def simATP(time = 20, dt = .001, ATPo = 1, Ro = .1, ADPo = 50, PGAo = 6, Into = 100, pqo = 40):
    tsteps = np.linspace(0, time, time/dt)
    ATP = np.zeros(len(tsteps))
    ATP[0] = ATPo
    ADP = np.zeros(len(tsteps))
    ADP[0] = ADPo
    PGA = np.zeros(len(tsteps))
    PGA[0] = PGAo
    Int = np.zeros(len(tsteps))
    Int[0] = Into
    R = np.zeros(len(tsteps))
    R[0] = Ro
    jpq = np.zeros(len(tsteps))
    jpq[0] = pqo
    NPQ = np.zeros(len(tsteps))
    for t in range(len(tsteps)-1):
        dDP, dInt, dADPr, dADPc = dADP(ATP[t], PGA[t], Int[t])
        dTP = dATP(ADP[t], dDP, jpq[t])
        ATP[t+1] = ATP[t] + dTP*dt
        ADP[t+1] = ADP[t] + (dDP-dTP)*dt
        Int[t+1] = Int[t] + dInt*dt
        dP, dRu = dPGA(R[t], dADPc)
        PGA[t+1] = PGA[t] + dP*dt
        dRub = dR(dRu, dADPr)
        R[t+1] = R[t] + dRub*dt
        tpq, f = PQ(jpq[t], dTP)
        jpq[t+1] = tpq
        NPQ[t+1] = f
    fig, axs = plt.subplots(4)
    fig.set_size_inches(18.5, 10.5)
    axs[0].plot(tsteps, ATP, label = 'ATP')
    axs[1].plot(tsteps[0:100], jpq[0:100], label = 'jpq')
    axs[1].plot(tsteps[0:100], NPQ[0:100], label = 'NPQ')
    axs[2].plot(tsteps, PGA, label = 'PGA')
    axs[3].plot(tsteps, R, label='RuBP')
    print(R[-1])
    for ax in axs:
        ax.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(IntSlider(value=20, description='time', max=60, min=-20), FloatSlider(value=0.001, descr…